In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from skimage import io,transform

import time

import os
import fnmatch

Using TensorFlow backend.


In [3]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

def combine_predictions_2(x,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4):#if cloud > thresh4, 
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),:] = 0
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),14] = 1
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [3]:
df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

np.random.shuffle(df_train.values)

train_values = df_train.values[:36000]
val_values = df_train.values[36000:]

x_train = np.zeros((36000,200,200,3), np.float32)
x_val = np.zeros((40479-36000,200,200,3), np.float32)
y_train = []
y_val = []

i=0

for f, tags in tqdm(train_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

i=0

for f, tags in tqdm(val_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_val[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_val.append(targets)
  
y_train = np.array(y_train, np.uint8)
y_val = np.array(y_val, np.uint8)


100%|██████████| 4479/4479 [00:21<00:00, 212.20it/s]


In [4]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean
x_val -= train_mean

In [5]:
#weather classifier (last four labels - mutually exclusive)
#x_train, x_val, y_train_w, y_val_w = train_test_split(x_train,y_train[:,-4:],test_size=0.1)
y_train_w = y_train[:,-4:]
y_val_w = y_val[:,-4:]
print(x_train.shape)
print(y_train_w.shape)
print(x_val.shape)
print(y_val_w.shape)

(36000, 200, 200, 3)
(36000, 4)
(4479, 200, 200, 3)
(4479, 4)


In [8]:
model = Sequential()#using same architecture for all three models
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(200, 200, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 64), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 705s - loss: 0.4580 - acc: 0.8435 - val_loss: 0.3631 - val_acc: 0.8598
Epoch 2/10
563/562 [==============================] - 703s - loss: 0.3364 - acc: 0.8829 - val_loss: 0.7531 - val_acc: 0.7459
Epoch 3/10
563/562 [==============================] - 704s - loss: 0.3159 - acc: 0.8867 - val_loss: 0.2698 - val_acc: 0.8951
Epoch 4/10
563/562 [==============================] - 704s - loss: 0.2914 - acc: 0.8968 - val_loss: 0.2871 - val_acc: 0.8993
Epoch 5/10
563/562 [==============================] - 704s - loss: 0.2882 - acc: 0.9002 - val_loss: 0.4651 - val_acc: 0.8651
Epoch 6/10
563/562 [==============================] - 704s - loss: 0.2817 - acc: 0.9013 - val_loss: 0.5382 - val_acc: 0.8542


In [10]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.89351364569
thresh: 0.1 	F2 score: 0.901145823366
thresh: 0.15 	F2 score: 0.899711882967
thresh: 0.2 	F2 score: 0.895964235214
thresh: 0.25 	F2 score: 0.8921049554
thresh: 0.3 	F2 score: 0.883232864479
thresh: 0.35 	F2 score: 0.875009302672


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [11]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 64), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 706s - loss: 0.2483 - acc: 0.9104 - val_loss: 0.2089 - val_acc: 0.9225
Epoch 2/10
563/562 [==============================] - 705s - loss: 0.2392 - acc: 0.9146 - val_loss: 0.2287 - val_acc: 0.9181
Epoch 3/10
563/562 [==============================] - 706s - loss: 0.2362 - acc: 0.9144 - val_loss: 0.2584 - val_acc: 0.9011
Epoch 4/10
563/562 [==============================] - 706s - loss: 0.2272 - acc: 0.9185 - val_loss: 0.2289 - val_acc: 0.9181


In [12]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.931158900265
thresh: 0.1 	F2 score: 0.941688195707
thresh: 0.15 	F2 score: 0.944084298153
thresh: 0.2 	F2 score: 0.942174592543
thresh: 0.25 	F2 score: 0.940207741949
thresh: 0.3 	F2 score: 0.936592989507
thresh: 0.35 	F2 score: 0.932313760512


In [13]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 64), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 708s - loss: 0.2095 - acc: 0.9222 - val_loss: 0.1896 - val_acc: 0.9283
Epoch 2/10
563/562 [==============================] - 706s - loss: 0.2059 - acc: 0.9236 - val_loss: 0.1922 - val_acc: 0.9259
Epoch 3/10
563/562 [==============================] - 706s - loss: 0.2005 - acc: 0.9264 - val_loss: 0.1894 - val_acc: 0.9283
Epoch 4/10
563/562 [==============================] - 706s - loss: 0.2042 - acc: 0.9250 - val_loss: 0.1988 - val_acc: 0.9203
Epoch 5/10
563/562 [==============================] - 706s - loss: 0.2014 - acc: 0.9260 - val_loss: 0.1774 - val_acc: 0.9299
Epoch 6/10
563/562 [==============================] - 706s - loss: 0.2017 - acc: 0.9257 - val_loss: 0.1883 - val_acc: 0.9312
Epoch 7/10
563/562 [==============================] - 706s - loss: 0.1968 - acc: 0.9272 - val_loss: 0.2043 - val_acc: 0.9254
Epoch 8/10
563/562 [==============================] - 706s - loss: 0.1959 - acc: 0.9279 - val_loss: 0.2417 - val_acc: 0.9087


In [14]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.923089497018
thresh: 0.1 	F2 score: 0.932467919072
thresh: 0.15 	F2 score: 0.934344400855
thresh: 0.2 	F2 score: 0.932919763127
thresh: 0.25 	F2 score: 0.930155540671
thresh: 0.3 	F2 score: 0.927598634899
thresh: 0.35 	F2 score: 0.921597082682


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [15]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 64), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 708s - loss: 0.1911 - acc: 0.9299 - val_loss: 0.1740 - val_acc: 0.9306
Epoch 2/10
563/562 [==============================] - 705s - loss: 0.1869 - acc: 0.9310 - val_loss: 0.1758 - val_acc: 0.9315
Epoch 3/10
563/562 [==============================] - 706s - loss: 0.1899 - acc: 0.9297 - val_loss: 0.1725 - val_acc: 0.9348
Epoch 4/10
563/562 [==============================] - 706s - loss: 0.1854 - acc: 0.9315 - val_loss: 0.1823 - val_acc: 0.9308
Epoch 5/10
563/562 [==============================] - 706s - loss: 0.1867 - acc: 0.9309 - val_loss: 0.1726 - val_acc: 0.9332
Epoch 6/10
563/562 [==============================] - 706s - loss: 0.1832 - acc: 0.9324 - val_loss: 0.1730 - val_acc: 0.9339


In [16]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.942299514135
thresh: 0.1 	F2 score: 0.951243368524
thresh: 0.15 	F2 score: 0.953550431112
thresh: 0.2 	F2 score: 0.955193017149
thresh: 0.25 	F2 score: 0.953305903741
thresh: 0.3 	F2 score: 0.951599527956
thresh: 0.35 	F2 score: 0.948016670388


In [17]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 64), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 709s - loss: 0.1812 - acc: 0.9323 - val_loss: 0.1735 - val_acc: 0.9337
Epoch 2/10
563/562 [==============================] - 706s - loss: 0.1811 - acc: 0.9336 - val_loss: 0.1716 - val_acc: 0.9344
Epoch 3/10
563/562 [==============================] - 706s - loss: 0.1804 - acc: 0.9330 - val_loss: 0.1737 - val_acc: 0.9330
Epoch 4/10
563/562 [==============================] - 706s - loss: 0.1787 - acc: 0.9345 - val_loss: 0.1720 - val_acc: 0.9339
Epoch 5/10
563/562 [==============================] - 706s - loss: 0.1790 - acc: 0.9333 - val_loss: 0.1744 - val_acc: 0.9326


In [18]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.944508234193
thresh: 0.1 	F2 score: 0.953499930894
thresh: 0.15 	F2 score: 0.954725225656
thresh: 0.2 	F2 score: 0.954066064917
thresh: 0.25 	F2 score: 0.952668006251
thresh: 0.3 	F2 score: 0.948511040942
thresh: 0.35 	F2 score: 0.944928183374


In [19]:
model.save("simple_200_weather")
model.save_weights("simple_200_weather_weights")

In [3]:
x_val = []
x_train = []
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

np.random.shuffle(df_train.values)

train_values = df_train.values[:36000]
val_values = df_train.values[36000:]

x_train = np.zeros((36000,200,200,3), np.float32)
x_val = np.zeros((40479-36000,200,200,3), np.float32)
y_train = []
y_val = []

i=0

for f, tags in tqdm(train_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

i=0

for f, tags in tqdm(val_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_val[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_val.append(targets)
  
y_train = np.array(y_train, np.uint8)
y_val = np.array(y_val, np.uint8)


100%|██████████| 4479/4479 [00:44<00:00, 100.87it/s]


In [4]:
y_train_2 = np.zeros((y_train.shape[0],7))
y_train_2[:,1:] = y_train[:,7:13]
y_train_2[:,0] = (np.sum(y_train[:,:7],axis=1)>0)
y_train_2 = np.array(y_train_2,np.uint8)

y_val_2 = np.zeros((y_val.shape[0],7))
y_val_2[:,1:] = y_val[:,7:13]
y_val_2[:,0] = (np.sum(y_val[:,:7],axis=1)>0)
y_val_2 = np.array(y_val_2,np.uint8)

In [5]:
print(y_train[100,:])
print(y_train_2[100,:])

[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0]
[0 0 0 0 1 0 1]


In [6]:
train_mean = np.load('simple_200_train_mean.npy')
x_train -= train_mean
x_val -= train_mean
#x_train, x_val, y_train_2, y_val_2 = train_test_split(x_train,y_train_2,test_size=0.1)
print(x_train.shape)
print(y_train_2.shape)
print(x_val.shape)
print(y_val_2.shape)

(36000, 200, 200, 3)
(36000, 7)
(4479, 200, 200, 3)
(4479, 7)


In [ ]:
#model for the more common classes + 1 class for others
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(200,200,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(2048))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(1024))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(7, activation='sigmoid'))

In [ ]:
model1.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 745s - loss: 0.2885 - acc: 0.8823 - val_loss: 0.2816 - val_acc: 0.8858
Epoch 2/10
563/562 [==============================] - 732s - loss: 0.2423 - acc: 0.9017 - val_loss: 0.3013 - val_acc: 0.8828
Epoch 4/10
563/562 [==============================] - 732s - loss: 0.2284 - acc: 0.9084 - val_loss: 0.2346 - val_acc: 0.9047
Epoch 5/10
563/562 [==============================] - 733s - loss: 0.2161 - acc: 0.9130 - val_loss: 0.2199 - val_acc: 0.9105
Epoch 6/10
563/562 [==============================] - 733s - loss: 0.2090 - acc: 0.9168 - val_loss: 0.2006 - val_acc: 0.9178
Epoch 7/10
563/562 [==============================] - 734s - loss: 0.2027 - acc: 0.9195 - val_loss: 0.2327 - val_acc: 0.8990
Epoch 8/10
563/562 [==============================] - 734s - loss: 0.1981 - acc: 0.9217 - val_loss: 0.2090 - val_acc: 0.9150
Epoch 9/10
563/562 [==============================] - 734s - loss: 0.1946 - acc: 0.9228 - val_loss: 0.2672 - val_acc: 0.8788


In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.82424984073
thresh: 0.1 	F2 score: 0.837270896841
thresh: 0.15 	F2 score: 0.833254790366
thresh: 0.2 	F2 score: 0.825835942826
thresh: 0.25 	F2 score: 0.815065760238
thresh: 0.3 	F2 score: 0.802570581189
thresh: 0.35 	F2 score: 0.790765886788


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [ ]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 735s - loss: 0.1855 - acc: 0.9261 - val_loss: 0.1826 - val_acc: 0.9283
Epoch 2/10
563/562 [==============================] - 734s - loss: 0.1822 - acc: 0.9279 - val_loss: 0.1713 - val_acc: 0.9323
Epoch 3/10
563/562 [==============================] - 734s - loss: 0.1794 - acc: 0.9289 - val_loss: 0.1769 - val_acc: 0.9273
Epoch 4/10
563/562 [==============================] - 733s - loss: 0.1769 - acc: 0.9300 - val_loss: 0.1748 - val_acc: 0.9315
Epoch 5/10
563/562 [==============================] - 734s - loss: 0.1754 - acc: 0.9310 - val_loss: 0.1665 - val_acc: 0.9333
Epoch 6/10
563/562 [==============================] - 737s - loss: 0.1736 - acc: 0.9307 - val_loss: 0.1655 - val_acc: 0.9340
Epoch 7/10
563/562 [==============================] - 739s - loss: 0.1723 - acc: 0.9316 - val_loss: 0.1710 - val_acc: 0.9298
Epoch 8/10
563/562 [==============================] - 739s - loss: 0.1703 - acc: 0.9322 - val_loss: 0.1695 - val_acc: 0.9309


NameError: name 'model1' is not defined

In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.845798008543
thresh: 0.1 	F2 score: 0.862875233777
thresh: 0.15 	F2 score: 0.86693817779
thresh: 0.2 	F2 score: 0.869761844756
thresh: 0.25 	F2 score: 0.866331314693
thresh: 0.3 	F2 score: 0.862697797486
thresh: 0.35 	F2 score: 0.859893670385


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [ ]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 739s - loss: 0.1609 - acc: 0.9364 - val_loss: 0.1601 - val_acc: 0.9359
Epoch 2/10
563/562 [==============================] - 733s - loss: 0.1594 - acc: 0.9367 - val_loss: 0.1568 - val_acc: 0.9374
Epoch 3/10
563/562 [==============================] - 732s - loss: 0.1586 - acc: 0.9375 - val_loss: 0.1563 - val_acc: 0.9364
Epoch 4/10
563/562 [==============================] - 733s - loss: 0.1579 - acc: 0.9375 - val_loss: 0.1563 - val_acc: 0.9378
Epoch 5/10
172/562 [========>.....................] - ETA: 487s - loss: 0.1588 - acc: 0.9382

In [ ]:
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                   epochs=5, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/5
  7/562 [..............................] - ETA: 728s - loss: 0.1564 - acc: 0.9340

In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

In [ ]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

In [ ]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 64), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

In [ ]:
y_pred = model1.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score        

In [ ]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_2,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=['others']+labels[7:13])

In [ ]:
num_rare = np.sum(np.sum(y_train[:,:7],axis=1)>0)
print(num_rare)

In [ ]:
model1.save("simple_200_major")
model1.save_weights("simple_200_major_weights")

In [4]:
num_rare = 2180
x_val = []
x_train = []
x_train = np.zeros((num_rare,200,200,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    if(np.sum(targets[:7])>0):
        x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
        i+=1
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:56<00:00, 715.90it/s]

(2180, 200, 200, 3)
(2180, 17)


In [5]:
train_mean = np.load('simple_200_train_mean.npy')
x_train -= train_mean
x_train, x_val, y_train_3, y_val_3 = train_test_split(x_train,y_train[:,:7],test_size=0.1)
print(x_train.shape)
print(y_train_3.shape)
print(x_val.shape)
print(y_val_3.shape)

(1962, 200, 200, 3)
(1962, 7)
(218, 200, 200, 3)
(218, 7)


In [6]:
#model for the rarer classes
from keras.models import load_model
model2 = load_model("simple_200_major")
for layer in model2.layers[:-1]:
    layer.trainable = False
model2.layers[-1].trainable = True

In [7]:
model2.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 64), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 64, callbacks=callbacks,
                  )#more steps per epoch to compensate for fewer images

Epoch 1/10
307/306 [==============================] - 128s - loss: 0.4260 - acc: 0.8563 - val_loss: 0.2420 - val_acc: 0.9017
Epoch 2/10
307/306 [==============================] - 125s - loss: 0.2349 - acc: 0.9042 - val_loss: 0.2249 - val_acc: 0.9122
Epoch 3/10
307/306 [==============================] - 125s - loss: 0.2178 - acc: 0.9095 - val_loss: 0.2143 - val_acc: 0.9128
Epoch 4/10
307/306 [==============================] - 125s - loss: 0.2104 - acc: 0.9126 - val_loss: 0.2122 - val_acc: 0.9135
Epoch 5/10
307/306 [==============================] - 125s - loss: 0.2057 - acc: 0.9151 - val_loss: 0.2076 - val_acc: 0.9135
Epoch 6/10
307/306 [==============================] - 125s - loss: 0.2027 - acc: 0.9161 - val_loss: 0.2059 - val_acc: 0.9201
Epoch 7/10
307/306 [==============================] - 125s - loss: 0.2000 - acc: 0.9168 - val_loss: 0.2040 - val_acc: 0.9174
Epoch 8/10
307/306 [==============================] - 126s - loss: 0.1983 - acc: 0.9175 - val_loss: 0.1985 - val_acc: 0.9220


In [8]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.748415511604
thresh: 0.1 	F2 score: 0.783445530005
thresh: 0.15 	F2 score: 0.780312959212
thresh: 0.2 	F2 score: 0.77209264996
thresh: 0.25 	F2 score: 0.76379787389
thresh: 0.3 	F2 score: 0.732379496141
thresh: 0.35 	F2 score: 0.703109072375


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [9]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 124s - loss: 0.1870 - acc: 0.9216 - val_loss: 0.1980 - val_acc: 0.9194
Epoch 2/10
154/153 [==============================] - 120s - loss: 0.1865 - acc: 0.9221 - val_loss: 0.1975 - val_acc: 0.9187
Epoch 3/10
154/153 [==============================] - 118s - loss: 0.1849 - acc: 0.9229 - val_loss: 0.1970 - val_acc: 0.9174
Epoch 4/10
154/153 [==============================] - 119s - loss: 0.1849 - acc: 0.9223 - val_loss: 0.1990 - val_acc: 0.9174
Epoch 5/10
154/153 [==============================] - 117s - loss: 0.1850 - acc: 0.9223 - val_loss: 0.1992 - val_acc: 0.9155
Epoch 6/10
154/153 [==============================] - 120s - loss: 0.1850 - acc: 0.9216 - val_loss: 0.1963 - val_acc: 0.9194
Epoch 7/10
154/153 [==============================] - 120s - loss: 0.1838 - acc: 0.9232 - val_loss: 0.1958 - val_acc: 0.9194
Epoch 8/10
154/153 [==============================] - 121s - loss: 0.1834 - acc: 0.9233 - val_loss: 0.1983 - val_acc: 0.9201


In [10]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.763513245165
thresh: 0.1 	F2 score: 0.792408852615
thresh: 0.15 	F2 score: 0.786238035665
thresh: 0.2 	F2 score: 0.771118789467
thresh: 0.25 	F2 score: 0.757592305757
thresh: 0.3 	F2 score: 0.755315275957
thresh: 0.35 	F2 score: 0.714831804281


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [11]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 123s - loss: 0.1815 - acc: 0.9231 - val_loss: 0.1962 - val_acc: 0.9201
Epoch 2/10
154/153 [==============================] - 121s - loss: 0.1802 - acc: 0.9245 - val_loss: 0.1967 - val_acc: 0.9194
Epoch 3/10
154/153 [==============================] - 122s - loss: 0.1815 - acc: 0.9235 - val_loss: 0.1964 - val_acc: 0.9194
Epoch 4/10
154/153 [==============================] - 121s - loss: 0.1824 - acc: 0.9228 - val_loss: 0.1959 - val_acc: 0.9187
Epoch 5/10
154/153 [==============================] - 120s - loss: 0.1814 - acc: 0.9233 - val_loss: 0.1960 - val_acc: 0.9187
Epoch 6/10
154/153 [==============================] - 122s - loss: 0.1805 - acc: 0.9238 - val_loss: 0.1961 - val_acc: 0.9207
Epoch 7/10
154/153 [==============================] - 122s - loss: 0.1819 - acc: 0.9227 - val_loss: 0.1960 - val_acc: 0.9194


In [12]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.764268669659
thresh: 0.1 	F2 score: 0.79180815362
thresh: 0.15 	F2 score: 0.785364291672
thresh: 0.2 	F2 score: 0.773958457445
thresh: 0.25 	F2 score: 0.759339793743
thresh: 0.3 	F2 score: 0.748216106014
thresh: 0.35 	F2 score: 0.714831804281


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [13]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 124s - loss: 0.1799 - acc: 0.9246 - val_loss: 0.1968 - val_acc: 0.9194
Epoch 2/10
154/153 [==============================] - 120s - loss: 0.1804 - acc: 0.9238 - val_loss: 0.1969 - val_acc: 0.9194
Epoch 3/10
154/153 [==============================] - 122s - loss: 0.1800 - acc: 0.9244 - val_loss: 0.1966 - val_acc: 0.9194
Epoch 4/10
154/153 [==============================] - 121s - loss: 0.1810 - acc: 0.9240 - val_loss: 0.1963 - val_acc: 0.9194
Epoch 5/10
154/153 [==============================] - 121s - loss: 0.1805 - acc: 0.9241 - val_loss: 0.1962 - val_acc: 0.9187
Epoch 6/10
154/153 [==============================] - 122s - loss: 0.1798 - acc: 0.9235 - val_loss: 0.1965 - val_acc: 0.9187
Epoch 7/10
154/153 [==============================] - 121s - loss: 0.1791 - acc: 0.9247 - val_loss: 0.1958 - val_acc: 0.9187
Epoch 8/10
154/153 [==============================] - 122s - loss: 0.1804 - acc: 0.9240 - val_loss: 0.1959 - val_acc: 0.9194


In [14]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.762530283172
thresh: 0.1 	F2 score: 0.784772694176
thresh: 0.15 	F2 score: 0.788367786648
thresh: 0.2 	F2 score: 0.772081893642
thresh: 0.25 	F2 score: 0.760322755736
thresh: 0.3 	F2 score: 0.752038735984
thresh: 0.35 	F2 score: 0.717889908257


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [16]:
y_pred = model2.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score    

thresh: 0.05 	F2 score: 0.763941015661
thresh: 0.1 	F2 score: 0.78436312668
thresh: 0.15 	F2 score: 0.7818420112
thresh: 0.2 	F2 score: 0.76978831566
thresh: 0.25 	F2 score: 0.760104319737
thresh: 0.3 	F2 score: 0.745158002039
thresh: 0.35 	F2 score: 0.717125382263


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [17]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_3,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels[:7])

,FN,FP,TN,TP,Total
blow_down,5,16,194,3,8
bare_ground,4,70,58,86,90
conventional_mine,1,20,191,6,7
blooming,0,29,161,28,28
artisinal_mine,2,27,151,38,40
selective_logging,1,60,123,34,35
slash_burn,2,31,168,17,19


In [18]:
model2.save("simple_200_rare")
model2.save_weights("simple_200_rare_weights")

In [19]:
#F2 score on training set
x_val = []
x_train = []
x_train = np.zeros((40479,200,200,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:16<00:00, 532.36it/s]

(40479, 200, 200, 3)
(40479, 17)


In [20]:
x_train -= train_mean
model = Sequential()#using same architecture for all three models
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(200, 200, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))
model.load_weights('simple_200_weather_weights')
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(200,200,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(2048))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(1024))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(7, activation='sigmoid'))
model1.load_weights('simple_200_major_weights')

In [21]:
y = model.predict(x_train,batch_size=128)
y1 = model1.predict(x_train,batch_size=128)
y2 = model2.predict(x_train,batch_size=128)

In [22]:
print(y.shape)
print(y1.shape)
print(y2.shape)

(40479, 4)
(40479, 7)
(40479, 7)


In [23]:
def combine_predictions_2(y,y1,y2,thresh):
    y_pred = np.zeros((y.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh[0:4]),np.uint8)
    y1_bool = np.array((y1[:,1:] > thresh[4:10]),np.uint8)
    y2_bool = np.array((y2 > thresh[10:17])*np.tile(y1[:,0]>thresh[17],(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool
    y_pred[:,13:] = y_bool
    y_pred[((np.nonzero(y[:,1]>thresh[18]))[0]),:] = 0
    y_pred[((np.nonzero(y[:,1]>thresh[18]))[0]),14] = 1
    return y_pred

def optimise_f2_thresholds(x_init,y, y1,y2, ytrue, num_thresh=19, verbose=True, resolution=100):
  def mf(x):
    p2 = combine_predictions_2(y, y1,y2,x)
    score = fbeta_score(ytrue, p2, beta=2, average='samples')
    return score

  x = list(x_init)
  for i in range(num_thresh):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

In [24]:
best_thresh = optimise_f2_thresholds([0.2]*19,y,y1,y2,y_train)

0 0.21 0.918700015484
1 0.09 0.918923133532
2 0.22 0.918955179887
3 0.12 0.919173172716
4 0.18 0.919250278888
5 0.24 0.919299402384
6 0.23 0.919335736141
7 0.19 0.919354111783
8 0.21 0.919373788484
9 0.14 0.919382161617
10 0.33 0.919425953016
11 0.3 0.91949832561
12 0.21 0.919505783726
13 0.6 0.919595727831
14 0.32 0.919617563736
15 0.52 0.919963089155
16 0.69 0.920055842168
17 0.23 0.920115614524
18 0.75 0.930827678634


In [25]:
best_thresh = optimise_f2_thresholds(best_thresh,y,y1,y2,y_train)

0 0.21 0.930827678634
1 0.09 0.930827678634
2 0.22 0.930827678634
3 0.12 0.930827678634
4 0.18 0.930827678634
5 0.24 0.930827678634
6 0.23 0.930827678634
7 0.21 0.930835409936
8 0.23 0.930838946472
9 0.22 0.930913670254
10 0.71 0.930914214303
11 0.3 0.930914214303
12 0.21 0.930914214303
13 0.35 0.930933458898
14 0.32 0.930933458898
15 0.51 0.93093402694
16 0.73 0.930936379718
17 0.23 0.930936379718
18 0.75 0.930936379718


In [26]:
best_thresh = optimise_f2_thresholds(best_thresh,y,y1,y2,y_train)

0 0.21 0.930936379718
1 0.09 0.930936379718
2 0.22 0.930936379718
3 0.12 0.930936379718
4 0.18 0.930936379718
5 0.24 0.930936379718
6 0.23 0.930936379718
7 0.21 0.930936379718
8 0.23 0.930936379718
9 0.22 0.930936379718
10 0.71 0.930936379718
11 0.3 0.930936379718
12 0.21 0.930936379718
13 0.35 0.930936379718
14 0.32 0.930936379718
15 0.51 0.930936379718
16 0.73 0.930936379718
17 0.23 0.930936379718
18 0.75 0.930936379718


In [27]:
print(best_thresh)
np.save('3net_200_best_thresh',best_thresh)

[0.21, 0.09, 0.22, 0.12, 0.18, 0.24, 0.23, 0.21, 0.23, 0.22, 0.71, 0.3, 0.21, 0.35, 0.32, 0.51, 0.73, 0.23, 0.75]


In [28]:
total,tp,tn,fp,fn = multilabelmetrics(y_train,combine_predictions_2(y,y1,y2,best_thresh))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,84,9,40372,14,98
bare_ground,445,722,38895,417,862
conventional_mine,26,105,40274,74,100
blooming,220,255,39892,112,332
artisinal_mine,59,147,39993,280,339
selective_logging,196,205,39934,144,340
slash_burn,205,2,40268,4,209
cultivation,976,3662,32340,3501,4477
habitation,568,1652,35167,3092,3660
road,602,2669,29740,7468,8070


In [32]:
#Test set
x_train = []
x_val = []
x_test = np.zeros((30000,200,200,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values[:30000], miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

x_test -= train_mean

y = model.predict(x_test,batch_size=128)
y1 = model1.predict(x_test,batch_size=128)
y2 = model2.predict(x_test,batch_size=128)



100%|██████████| 30000/30000 [05:47<00:00, 86.27it/s]

(30000, 200, 200, 3)


MemoryError: 

In [33]:
x_test=[]
x_test = np.zeros((31191,200,200,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values[30000:], miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

x_test -= train_mean

y_1 = model.predict(x_test,batch_size=128)
y1_1 = model1.predict(x_test,batch_size=128)
y2_1 = model2.predict(x_test,batch_size=128)

y = np.concatenate([y,y_1])
y1 = np.concatenate([y1,y1_1])
y2 = np.concatenate([y2,y2_1])

100%|██████████| 31191/31191 [02:03<00:00, 253.20it/s]

(31191, 200, 200, 3)


In [34]:
print(y.shape)
print(y1.shape)
print(y2.shape)

(61191, 4)
(61191, 7)
(61191, 7)


In [35]:
y_pred = combine_predictions_2(y,y1,y2,best_thresh)
print(y_pred.shape)

(61191, 17)


In [37]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_200_3net_1.csv', index=False)
#test set score:0.92387

In [70]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[1][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [71]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_2.csv', index=False)
#test set score:0.92445

In [72]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[2][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [73]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_3.csv', index=False)
#test set score:0.92446

In [74]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[3][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [75]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_4.csv', index=False)
#test set score: 0.92457

In [76]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[4][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [77]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_5.csv', index=False)
#test set score: 0.92455